In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
#Initialize file name
fname = "raw_whatsapp.txt"

In [3]:
#Open file
with open(fname, "r", encoding='utf-8') as data:
    #Read each line as lits item
    lines = data.read().splitlines()
    last_line = lines[-1]

#Remove all blank spaces
lines_no_na = list(filter(lambda a: a != "", lines))

In [4]:
#Loop through the messages to combine multi-line messages as one list item
formatted_lines = []
for index,ln in enumerate(lines_no_na):
    if index == 0:
        formatted_lines.append(ln)
        continue
    elif re.search("^\d{2}/\d{2}/\d{4}", ln):
        formatted_lines.append(ln)
    else:
        if len(formatted_lines) == 1:
            formatted_lines[0] = " ".join([formatted_lines[0],ln])
        else:
            formatted_lines[-1] = " ".join([formatted_lines[-1],ln])

In [13]:
#Ignore this, only required for cleaning original data
#formatted_lines.pop(38988)

#Change data structure of each entry to dictionary {"date":"abc","phone":"123","message":"xyz"}
formatted_dict_lst = []
for index,frmt_lst in enumerate(formatted_lines):
    temp_dict = {}
    #Check if date is at the start of entry, if yes extract, if no save entry data as np.nan
    if re.search("^\d{2}/\d{2}/\d{4}.*-", frmt_lst) is None:
        temp_dict["date"] = np.nan
    else:
        temp_dict["date"] = list(re.findall("^\d{2}/\d{2}/\d{4}.*- " , frmt_lst)[0].replace("-","").strip())
        temp_dict["date"] = temp_dict["date"][:18]
        temp_dict["date"] = "".join(temp_dict["date"]).strip(" ")
    #Check is entry has a phone entry using RegEx, if yes, extract, is no stave entry as np.nan
    if re.search("-.[+].*: ", frmt_lst) is None:
        temp_dict["phone"] = np.nan
    else:
        temp_dict["phone"] = re.findall("-.[+].*: ", frmt_lst)[0].strip("-+").replace(':', '').replace(' ','')
        if len(temp_dict["phone"])<=14:
            pass
        else:
            #if phone number entry is longer than 14 digit, check is extra digits are numbers, if yes OK, is no delete
            temp_dict["phone"] = temp_dict["phone"][:14]
            if temp_dict["phone"][-1] not in ["0","1","2","3","4","5","6","7","8","9"]:
                tempp = list(temp_dict["phone"])
                tempp.pop(-1)
                temp_dict["phone"]=tempp
                temp_dict["phone"] = "".join(temp_dict["phone"]).strip(" ")
            if temp_dict["phone"][-1] not in ["0","1","2","3","4","5","6","7","8","9"]:
                tempp.pop(-1)
                temp_dict["phone"]=tempp
                temp_dict["phone"] = "".join(temp_dict["phone"]).strip(" ")
    #Check for message entry and store in dictionary
    if re.search(" [+].*", frmt_lst):
        temp_dict["message"] = list(re.findall(" [+].*", frmt_lst)[0])
        del temp_dict["message"][:18]
        temp_dict["message"] = "".join(temp_dict["message"]).strip(":").replace("*","")
    else: 
        temp_dict["message"] = list(re.findall(".[:].*", frmt_lst)[0])
        del temp_dict["message"][:6]
        temp_dict["message"] = "".join(temp_dict["message"]).strip().replace("*","")
    #If message entry is "media omitted" store entry as np.nan
    if temp_dict["message"] == " <Media omitted>":
        temp_dict["message"] = np.nan
    #for every iteration, save dictionary as list item
    formatted_dict_lst.append(temp_dict)

In [14]:
#convert the list of dictionary to a PD dataframe
df = pd.DataFrame(formatted_dict_lst)

In [15]:
#Display first five terms of DATAFRAME
df.head()

,date,phone,message
0,"22/03/2019, 07:46",+61459847002,I am a cat
1,"22/03/2019, 07:49",+2347062612782,"Funny job, but I'll do it"
2,"23/03/2019, 07:52",+2348059872748,"Hahahahah, I don't like chasing stars"
3,"23/03/2019, 07:56",+2348069087511,You are saying funny abi things 🤪🤣😀😀
4,"23/03/2019, 07:57",+2348064487511,If you know how big i am...you won't talk lik...


In [16]:
# saving the dataframe 
df.to_csv('clean_whatsapp.csv') 